In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
from sklearn.datasets import load_digits

In [ ]:
import mlflow
import mlflow.xgboost
import xgboost as xgb


In [ ]:

# Séparation des données en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialiser et entraîner le modèle XGBoost
model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, use_label_encoder=False, eval_metric="mlogloss")
model.fit(X_train, y_train)

# Prédictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

# Calcul des métriques
accuracy = accuracy_score(y_test, y_pred)
logloss = log_loss(y_test, y_pred_proba)

# Suivi avec MLflow
mlflow.set_experiment("XGBoost_Exp")

with mlflow.start_run():
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("learning_rate", 0.1)
    
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("log_loss", logloss)

    # Enregistrer le modèle
    mlflow.xgboost.log_model(model, "xgboost_model")

    print(f"Modèle enregistré avec accuracy={accuracy:.4f} et log_loss={logloss:.4f}")

# Afficher l'interface MLflow
print("Lancer la commande suivante pour voir les résultats dans l'UI MLflow:")
print("mlflow ui --port 5000")
